In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

"""import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))"""

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd
import librosa
from tensorflow import keras

In [ ]:
import matplotlib.pyplot as plt
ravdess = "/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/"
crema = "/kaggle/input/cremad/AudioWAV/"


In [ ]:
import os
ravdess_dir_list=os.listdir(ravdess)
file_emotion=[]
file_path=[]

for dir in ravdess_dir_list:
    actor = os.listdir(ravdess + dir)
    for file in actor:
        part = file.split('.')[0]
        part = part.split('-')
        file_emotion.append(int(part[2]))
        file_path.append(ravdess + dir + '/' + file)

emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])
path_df = pd.DataFrame(file_path, columns=['Path'])
ravdess_df = pd.concat([emotion_df, path_df], axis=1)
ravdess_df.Emotions.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)
ravdess_df.head()

In [ ]:
crema_dir_list=os.listdir(crema)
file_emotion=[]
file_path=[]

for file in crema_dir_list:
    # storing file paths
    file_path.append(crema + file)
    # storing file emotions
    part=file.split('_')
    if part[2] == 'SAD':
        file_emotion.append('sad')
    elif part[2] == 'ANG':
        file_emotion.append('angry')
    elif part[2] == 'DIS':
        file_emotion.append('disgust')
    elif part[2] == 'FEA':
        file_emotion.append('fear')
    elif part[2] == 'HAP':
        file_emotion.append('happy')
    elif part[2] == 'NEU':
        file_emotion.append('neutral')
    else:
        file_emotion.append('Unknown')

# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
crema_df = pd.concat([emotion_df, path_df], axis=1)
crema_df.head()

In [ ]:
ex_path=ravdess_df['Path'][0]
aud,sr=librosa.load(ex_path)
print(aud.shape,aud.ndim,sr,librosa.get_duration(y=aud,sr=sr))
plt.plot(aud)

#plt.figure()
#librosa.display.waveplot(aud, sr=sr)
plt.show()

In [ ]:
df = pd.concat([ravdess_df, crema_df], axis = 0)
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
from sklearn.preprocessing import OneHotEncoder

df['Emotions']=df["Emotions"].map({'neutral':1, 'calm':2, 'happy':3,'sad' :4, 'angry':5, 'fear':6,'disgust':7, 'surprise':8,'Unknown':9})

X=df[['Path']]
y=df[['Emotions']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X.shape,X_train.shape,X_test.shape)

In [ ]:
#data augmentation on train_data
"""def stretch():
    new_aud=librosa.effects.time_stretch(aud,rate=0.8)
    
def shift():
    new_aud=librosa.effects.pitch_shift(aud,sr,0.7)

def check_duration():
    duration=librosa.get_duration(y=aud,sr=sr)
    """


In [ ]:
def create_mel(path):
    aud,sr=librosa.load(path)

    # Compute the Mel spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(y=aud, sr=sr)
    
    # Convert to decibels
    mel_spectrogram = librosa.amplitude_to_db(mel_spectrogram)
    return mel_spectrogram
    # Plot the Mel spectrogram
 mel_spectrogram=create_mel(ravdess_df['Path'][0])   
    
plt.figure(figsize=(10, 6))
librosa.display.specshow(mel_spectrogram, sr=sr, x_axis="time", y_axis="mel")
plt.colorbar(format="%+2.0f dB")
plt.title("Mel Spectrogram")
plt.show()



In [ ]:
# created df with path and mapped emotions
# function for creating mel spectrogram that takes path as input
# split X and y
# now, make batches , call create_mel , call feature extraction to create
# set fixed input , data augmentation on spec , input this to cnn model

In [ ]:
def pad_spectrogram(spectrogram, target_length):
    current_length = spectrogram.shape[1]
    if current_length >= target_length:
        return spectrogram[:, :target_length] #truncation
    else:
        pad_width = target_length - current_length
        return np.pad(spectrogram, ((0, 0), (0, pad_width)), mode='constant')



In [ ]:
def preprocess_audio(path, target_duration=4, sr=22050):
    mel_spec_db=create_mel(path)
    target_length=172 #duration*sr/hop_length , it is no. of time frames
    # Pad or truncate to fixed size
    mel_spec_fixed = pad_spectrogram(mel_spec_db, target_length)
    return mel_spec_fixed

In [ ]:
def batch_generator(df, batch_size, sr=22050):
    """
    Yields batches of Mel spectrograms and labels from a DataFrame.
    
    Parameters:
        df (pd.DataFrame): DataFrame with 'Path' column for audio file paths and 'Label' column for labels.
        batch_size (int): Number of samples per batch.
        n_mels (int): Number of Mel bands to generate.
        n_fft (int): Length of the FFT window.
        hop_length (int): Number of samples between successive frames.
        sr (int): Sampling rate.
    
    Yields:
        X_batch (np.ndarray): Batch of Mel spectrograms (shape: [batch_size, n_mels, time, 1]).
        y_batch (np.ndarray): Batch of labels (one-hot encoded).
    """
    num_samples = len(df)
    while True:  # Infinite loop for generator
        for start_idx in range(0, num_samples, batch_size):
            end_idx = min(start_idx + batch_size, num_samples)
            batch_df = df.iloc[start_idx:end_idx]
            
            X_batch = []
            y_batch = []
            
            for _, row in batch_df.iterrows():
                mel_spec_db=preprocess_audio(row['Path'])
                # Add a channel dimension (for CNN input)
                mel_spec_db = np.expand_dims(mel_spec_db, axis=-1)
                
                X_batch.append(mel_spec_db)
                y_batch.append(row['Emotions'])  # Assuming labels are already encoded
            
            # Convert to NumPy arrays
            X_batch = np.array(X_batch, dtype=np.float32)
            y_batch = np.array(y_batch, dtype=np.int32)
            
            yield X_batch, to_categorical(y_batch)  # One-hot encode labels


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(172, None, 1)),  # Variable time dimension
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(9, activation='softmax')  
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_gen, steps_per_epoch=len(X_train) // batch_size, epochs=10)

In [ ]:
model.summary()